In [85]:
import cv2
import numpy as np
from skimage.util import random_noise
from tqdm.notebook import tqdm
from skimage.transform import rotate
import os

In [86]:
def make_paper(image):
    paper = cv2.imread("paper/" + np.random.choice(os.listdir("paper/")))
    paper = cv2.resize(paper, (image.shape[1], image.shape[0]))
    
    mask = (image[:, :, 0] == 255) * (image[:, :, 1] == 255) * (image[:, :, 2] == 255)
    mask = np.stack([mask, mask, mask]).transpose((1, 2, 0))

    return paper * mask + image * (1 - mask)

def make_background(image):
    background = cv2.imread("backs/" + np.random.choice(os.listdir("backs/")))
    
    image_long_side = np.random.randint(background.shape[0] * 0.2, background.shape[0] * 0.5)
    image = cv2.resize(image, (int(image.shape[1] / image.shape[0] * image_long_side), image_long_side))
    
    h_pad = np.random.randint(0, background.shape[0] - image.shape[0])
    w_pad = np.random.randint(0, background.shape[1] - image.shape[1])
    image = np.pad(image, ((h_pad, background.shape[0] - image.shape[0] - h_pad), 
                           (w_pad, background.shape[1] - image.shape[1] - w_pad), 
                           (0, 0)))
    image = image * 255
    
    mask = (image[:, :, 0] == 0) * (image[:, :, 1] == 0) * (image[:, :, 2] == 0)
    mask = np.stack([mask, mask, mask]).transpose((1, 2, 0))

    return background * mask + image * (1 - mask)

In [87]:
def aug(image):
    image = make_paper(image) / 255
    image = rotate(image, np.random.uniform(0, 360), resize=True)
    image = make_background(image)
    image = cv2.blur(image, (np.random.randint(1, 5), np.random.randint(1, 5)))
    return image

In [89]:
for file in tqdm(os.listdir('images//')):
    image = cv2.imread('images/' + file)
    for i in tqdm(range(10)):
        cv2.imwrite('results/' + file[:-4]+ '_' + str(i) + '.png', aug(image))